#### **BigBasket Product Query Engine  Notebook 2**
**It contains**
1. importing vector embedding from file space.pkl which is python dic object vector DB 
2. storing vector in vector DB Qdrant 
3. saving qdrant for api
4. saving bert model for api
5. saving sentence transformer for api

In [1]:
!pip install -q qdrant-client
!pip install -q sentence-transformers
!pip install -q torch


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.1 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<4.0.0dev,>=3.12.0, but you have protobuf 4.25.1 which is incompatible.
google-cloud-bigtable 1.7.3 requires protobuf<4.0.0dev, but you have protobuf 4.25.1 which is incompatible.
google-cloud-datastore 

In [2]:
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import torch
import pickle
import numpy as np

In [3]:
VECTOR_SPACE_PATH = "/kaggle/input/chaabi/space.pkl"
with open(VECTOR_SPACE_PATH, 'rb') as file:
    vs = pickle.load(file)

In [4]:
client = QdrantClient(":memory:")

In [5]:
collection_name = "qdrant-space"

collections = client.get_collections()
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
print(client.get_collections())

collections=[CollectionDescription(name='qdrant-space')]


In [6]:
%%time

 
total_records = len(vs["payload"]) # total records data
_payload = vs["payload"]
_emb = vs["emb"]
ids = list(range(0,total_records))

batch_size = 2  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

client.upsert(
    collection_name=collection_name,
    points=models.Batch(ids=ids, vectors=_emb, payloads=_payload),
)


collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")


Vector count in collection: 27555
CPU times: user 2min, sys: 235 ms, total: 2min
Wall time: 2min


In [7]:
# dump qdrant space note keep sentence transformer in environment 

with open('qdrant_space_client.pkl', 'wb') as file:
    pickle.dump(client, file)

In [8]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever



SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [9]:
# dump encoder
import pickle

# Deserialize the data from the pickle file
with open('encodermodel.pkl', 'wb') as file:
    pickle.dump(retriever, file)

In [10]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# dump bert model 
# reader.save_pretrained("bert-question-answering")
with open('bert-question-answering.pkl', 'wb') as file:
    pickle.dump(reader, file)